# Vax vs. No vote

### Load python tools

In [3]:
%load_ext lab_black

In [4]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_latimes as lat
import jenkspy

In [5]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")

ThemeRegistry.enable('latimes')

In [6]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.options.display.max_colwidth = None

### FIPS codes for counties

In [7]:
fips = pd.read_html(
    "https://www.nrcs.usda.gov/wps/portal/nrcs/detail/ca/home/?cid=nrcs143_013697",
    attrs={"class": "data"},
)[0]

In [8]:
ca_fips = fips[fips["State"] == "CA"].copy()

In [9]:
ca_fips["FIPS"] = ca_fips["FIPS"].astype("str").str[1:]
ca_fips["Name"] = ca_fips["Name"].str.upper()

### Vax percentage

In [10]:
vax_url = "https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/cdph-vaccination-county-totals.csv"

In [18]:
vax_src = pd.read_csv(vax_url, dtype={"fips": str})

In [96]:
vax_slim = vax_src[vax_src["date"] == vax_df["date"].max()][
    ["county", "fips", "fully_vaccinated_percent", "population"]
]

In [97]:
vax_src.head()

,date,county,fips,population,doses_administered,new_doses_administered,pfizer_doses,new_pfizer_doses,moderna_doses,new_moderna_doses,jj_doses,new_jj_doses,partially_vaccinated,new_partially_vaccinated,at_least_one_dose,new_at_least_one_dose,fully_vaccinated,new_fully_vaccinated,partially_vaccinated_percent,at_least_one_dose_percent,fully_vaccinated_percent
0,2021-09-13,Alameda,001,1643700.0,2294079,1819,1468109,1320,714104,448,111866,51,96038,588,1239516,637,1143478,935,0.058428,0.754101,0.695673
1,2021-09-13,Alpine,003,1148.0,1525,0,33,0,1485,0,7,0,147,0,835,0,688,0,0.128049,0.727352,0.599303
2,2021-09-13,Amador,005,37829.0,37751,31,10305,12,26014,16,1432,3,6057,10,22617,14,16560,16,0.160115,0.597875,0.437759
3,2021-09-13,Butte,007,227075.0,197822,145,97917,58,91836,68,8069,19,12851,58,108358,76,95507,69,0.056594,0.477190,0.420597
4,2021-09-13,Calaveras,009,45235.0,42642,66,18152,39,23167,26,1323,1,4221,27,23956,31,19735,37,0.093313,0.529590,0.436277


### Yes/No vote

In [114]:
recall_src = pd.read_json(
    "data/processed/gov_race_change_counties.json", dtype={"fips": str}
)

In [115]:
recall_src.rename(columns={"d_pct_21": "no_percent"}, inplace=True)

In [116]:
recall_slim = recall_src[["county", "fips", "no_percent", "leader_2021"]]

In [117]:
recall_src.head()

,county,fips,dem_2018,dem_2021,rep_2018,rep_2021,votes_2018,votes_2021,d_pct_18,no_percent,r_pct_18,r_pct_21,d_change,r_change,leader_2021,leader_2018
0,Alameda,001,462558,287014,111677,60716,574235,347730,80.6,82.5,19.4,17.5,1.9,-1.9,D,D
1,Alpine,003,386,340,229,218,615,558,62.8,60.9,37.2,39.1,-1.9,1.9,D,D
2,Amador,005,6237,6411,11356,10971,17593,17382,35.5,36.9,64.5,63.1,1.4,-1.4,R,R
3,Butte,007,41500,31774,47226,35182,88726,66956,46.8,47.5,53.2,52.5,0.7,-0.7,R,R
4,Calaveras,009,7765,7106,13845,11450,21610,18556,35.9,38.3,64.1,61.7,2.4,-2.4,R,R


---

### Merge

In [118]:
merged = pd.merge(recall_slim, vax_slim, on=["fips", "county"])

In [119]:
merged.corr(method="pearson")

,no_percent,fully_vaccinated_percent,population
no_percent,1.000000,0.889498,0.275861
fully_vaccinated_percent,0.889498,1.000000,0.262278
population,0.275861,0.262278,1.000000


---

In [120]:
domain = ["R", "D"]
range_ = ["#d94f54", "#5789b8"]

alt.Chart(merged).mark_circle(size=100).encode(
    x=alt.X(
        "no_percent",
        scale=alt.Scale(domain=[10, 100]),
        axis=alt.Axis(tickCount=6, title="No percentage"),
    ),
    y=alt.Y(
        "fully_vaccinated_percent",
        scale=alt.Scale(domain=[0.2, 1]),
        axis=alt.Axis(tickCount=6, title="One dose percentage", format="%"),
    ),
    #     size="population",
    color=alt.Color("leader_2021", scale=alt.Scale(domain=domain, range=range_)),
    tooltip=["county", "population"],
).properties(width=1000, height=1000)

alt.Chart(...)

In [124]:
merged["fully_vaccinated_percent"] = round(merged["fully_vaccinated_percent"] * 100, 2)

In [125]:
merged.to_csv("data/processed/no_vax_corr.csv", index=False)